In [1]:
import os
import getpass
import json
import requests


In [2]:
api_key = getpass.getpass("Paste your GROQ API key here: ")
os.environ["GROQ_API_KEY"] = api_key
print("API key stored in environment (temporary).")


API key stored in environment (temporary).


In [3]:
import openai
openai.api_key = os.environ.get("GROQ_API_KEY")
openai.api_base = "https://api.groq.com/openai/v1"
print("Client configured with Groq base:", openai.api_base)

Client configured with Groq base: https://api.groq.com/openai/v1


In [4]:
def send_groq_request(messages, model="llama-3.3-70b-versatile", max_tokens=50):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {"model": model, "messages": messages, "max_tokens": max_tokens}
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

In [5]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Say 'hello' and confirm you're reachable."}
]
resp = send_groq_request(messages)
print(resp["choices"][0]["message"]["content"])

Hello. Yes, I'm reachable and ready to assist you. How can I help you today?


In [6]:
conversation_history = []

def add_message(role, content):
    conversation_history.append({"role": role, "content": content})

def truncate_by_turns(n):
    return conversation_history[-n:]

def truncate_by_length(max_chars):
    total = 0
    truncated = []
    for msg in reversed(conversation_history):
        if total + len(msg["content"]) <= max_chars:
            truncated.insert(0, msg)
            total += len(msg["content"])
        else:
            break
    return truncated

In [7]:
add_message("user", "Hello, can you help me with Python?")
add_message("assistant", "Sure! What do you need help with?")
add_message("user", "Explain lists and dictionaries.")
add_message("assistant", "Lists are ordered collections; dictionaries are key-value pairs.")


In [8]:
print("Last 2 turns:", truncate_by_turns(2))
print("Truncated by 50 chars:", truncate_by_length(50))

Last 2 turns: [{'role': 'user', 'content': 'Explain lists and dictionaries.'}, {'role': 'assistant', 'content': 'Lists are ordered collections; dictionaries are key-value pairs.'}]
Truncated by 50 chars: []


In [9]:
def summarize_conversation():
    if not conversation_history:
        return []
    messages = [
        {"role": "system", "content": "Summarize this conversation concisely."},
        {"role": "user", "content": json.dumps(conversation_history)}
    ]
    summary = send_groq_request(messages, max_tokens=200)
    return [{"role": "system", "content": "Summary: " + summary["choices"][0]["message"]["content"]}]


In [10]:
def add_message_with_summary(role, content, k=3):
    add_message(role, content)
    if len(conversation_history) % k == 0:
        summary = summarize_conversation()
        conversation_history.clear()
        conversation_history.extend(summary)
        print(f"--- Conversation summarized after {k} messages ---")

In [11]:
conversation_history.clear()
add_message_with_summary("user", "Hi, I want to learn Python basics.", k=3)
add_message_with_summary("assistant", "Let's start with variables.", k=3)
add_message_with_summary("user", "Can you also explain lists?", k=3)
add_message_with_summary("assistant", "Lists are ordered collections in Python.", k=3)
add_message_with_summary("user", "What about dictionaries?", k=3)
add_message_with_summary("assistant", "Dictionaries store key-value pairs.", k=3)

--- Conversation summarized after 3 messages ---
--- Conversation summarized after 3 messages ---


In [12]:
print("\nCurrent conversation history:")
for msg in conversation_history:
    print(f"{msg['role']}: {msg['content']}")


Current conversation history:
system: Summary: A user is learning Python basics, starting with variables and lists, and has now asked about dictionaries.
assistant: Dictionaries store key-value pairs.


In [13]:
function_schema = {
    "name": "extract_user_info",
    "description": "Extract name, email, phone, location, and age from chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

In [14]:
def extract_user_info(chat_text):
    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": [{"role": "user", "content": chat_text}],
        "functions": [function_schema],
        "function_call": {"name": "extract_user_info"},
        "max_tokens": 200
    }
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}", "Content-Type": "application/json"}
    resp = requests.post(url, headers=headers, json=payload)
    func_args = resp.json()["choices"][0]["message"]["function_call"]["arguments"]
    return json.loads(func_args)

In [15]:
sample_chat = "Hi, I'm Shubham Khedekar. My email is shubham@example.com, phone 9876543210, location Pune, age 24."
info = extract_user_info(sample_chat)
print(json.dumps(info, indent=2))

{
  "age": 24,
  "email": "shubham@example.com",
  "location": "Pune",
  "name": "Shubham Khedekar",
  "phone": "9876543210"
}


In [16]:
sample_chats = [
    "Hi, I'm Shubham Khedekar. My email is shubham@example.com, phone is 9876543210, I live in Pune, and I’m 24.",
    "Hello, my name is Priya Singh, I’m 30, based in Bangalore. You can reach me at priya@abc.com or 9123456789.",
    "This is Rohan Sharma, age 28, location Mumbai. Contact: rohan@gmail.com, 9988776655."
]

for chat in sample_chats:
    info = extract_user_info(chat)
    print(json.dumps(info, indent=2))


{
  "age": 24,
  "email": "shubham@example.com",
  "location": "Pune",
  "name": "Shubham Khedekar",
  "phone": "9876543210"
}
{
  "age": 30,
  "email": "priya@abc.com",
  "location": "Bangalore",
  "name": "Priya Singh",
  "phone": "9123456789"
}
{
  "age": 28,
  "email": "rohan@gmail.com",
  "location": "Mumbai",
  "name": "Rohan Sharma",
  "phone": "9988776655"
}
